In [81]:
import pandas as pd

df = pd.read_csv('data/ex2-2.csv', sep=';')

df['<DATE>'] = pd.to_datetime(df['<DATE>'], format='%d/%m/%y')

df[['<HOUR>', '<MINUTE>']] = df['<TIME>'].str.split(':', 1, expand=True)
df = df.drop(columns=['<TICKER>', '<PER>', '<VOL>', '<TIME>'])
#df = df.set_index(['<DATE>', '<HOUR>'])
df.has_nulls = df.isnull().values.any()

if df.has_nulls:
    raise Exception('There are nulls')

df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<HOUR>,<MINUTE>
0,2018-09-03,107900.0,108060.0,107900.0,108030.0,10,01
1,2018-09-03,107960.0,107960.0,107960.0,107960.0,10,02
2,2018-09-03,108000.0,108000.0,108000.0,108000.0,10,04
3,2018-09-03,108000.0,108000.0,108000.0,108000.0,10,05
4,2018-09-03,108080.0,108080.0,108080.0,108080.0,10,08
...,...,...,...,...,...,...,...
58965,2018-12-20,109380.0,109380.0,109380.0,109380.0,18,40
58966,2018-12-20,109380.0,109390.0,109380.0,109390.0,18,41
58967,2018-12-20,109380.0,109390.0,109380.0,109390.0,18,43
58968,2018-12-20,109390.0,109390.0,109390.0,109390.0,18,44


In [82]:
'''
Transform the data to hour intervals and make a Japanease Candlestick chart for the following date: September 28.
What chart from the file corresponds to the chart that you made? Enter the chart's number:
'''
from collections import OrderedDict

day_df = df.loc[(df['<DATE>'].dt.month == 9) & (df['<DATE>'].dt.day == 28)]
hours = list(OrderedDict.fromkeys(day_df['<HOUR>'].tolist()))
day_df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<HOUR>,<MINUTE>
11567,2018-09-28,116980.0,117260.0,116760.0,117160.0,10,01
11568,2018-09-28,117150.0,117170.0,116850.0,116930.0,10,02
11569,2018-09-28,116950.0,116960.0,116860.0,116930.0,10,03
11570,2018-09-28,116920.0,116990.0,116880.0,116920.0,10,04
11571,2018-09-28,116930.0,117010.0,116870.0,116950.0,10,05
...,...,...,...,...,...,...,...
12372,2018-09-28,117610.0,117620.0,117590.0,117620.0,23,46
12373,2018-09-28,117620.0,117630.0,117610.0,117620.0,23,47
12374,2018-09-28,117610.0,117620.0,117580.0,117600.0,23,48
12375,2018-09-28,117610.0,117690.0,117560.0,117670.0,23,49


In [83]:
#df['<OPEN>'].max().sum()
#df['<OPEN>'].loc[(df['<MINUTE>'] == df['<MINUTE>'].min()) & (df['<HOUR>'] == 10)]
cols = ['<DATE>','<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<HOUR>']
converted_df = pd.DataFrame(columns=cols)

for hour in hours:
    hour_df = day_df.loc[day_df['<HOUR>'] == hour]
    converted_df = pd.concat(
        [
            converted_df, 
            pd.DataFrame(
                #columns=cols,
                data=[{
                    '<DATE>': '2018-09-28',
                    '<OPEN>': hour_df['<OPEN>'].loc[hour_df['<MINUTE>'] == hour_df['<MINUTE>'].min()].sum(), 
                    '<HIGH>': hour_df['<HIGH>'].max().sum(), 
                    '<LOW>': hour_df['<LOW>'].min().sum(), 
                    '<CLOSE>': hour_df['<CLOSE>'].loc[hour_df['<MINUTE>'] == hour_df['<MINUTE>'].max()].sum(), 
                    '<HOUR>': hour
                }]
            )
        ]
    )
converted_df.head(len(converted_df))

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<HOUR>
0,2018-09-28,116980.0,117260.0,116300.0,116790.0,10
0,2018-09-28,116780.0,117530.0,116640.0,117510.0,11
0,2018-09-28,117510.0,117650.0,117010.0,117100.0,12
0,2018-09-28,117090.0,117240.0,116350.0,116860.0,13
0,2018-09-28,116860.0,117180.0,116510.0,116900.0,14
0,2018-09-28,116900.0,117270.0,116620.0,117210.0,15
0,2018-09-28,117210.0,117230.0,116540.0,116930.0,16
0,2018-09-28,116950.0,117880.0,116790.0,117830.0,17
0,2018-09-28,117840.0,118260.0,117720.0,118170.0,18
0,2018-09-28,118130.0,118350.0,117720.0,117970.0,19


In [84]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Candlestick(
            x=converted_df['<HOUR>'],
            open=converted_df['<OPEN>'],
            high=converted_df['<HIGH>'],
            low=converted_df['<LOW>'],
            close=converted_df['<CLOSE>']
        )
    ]
)

with open('output/plot.png', 'wb') as file:
    file.write(fig.to_image(format='png'))
    file.close()

with open('output/plot.html', 'w') as file:
    file.write(fig.to_html())
    file.close()

In [85]:
day_df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<HOUR>,<MINUTE>
11567,2018-09-28,116980.0,117260.0,116760.0,117160.0,10,01
11568,2018-09-28,117150.0,117170.0,116850.0,116930.0,10,02
11569,2018-09-28,116950.0,116960.0,116860.0,116930.0,10,03
11570,2018-09-28,116920.0,116990.0,116880.0,116920.0,10,04
11571,2018-09-28,116930.0,117010.0,116870.0,116950.0,10,05
...,...,...,...,...,...,...,...
12372,2018-09-28,117610.0,117620.0,117590.0,117620.0,23,46
12373,2018-09-28,117620.0,117630.0,117610.0,117620.0,23,47
12374,2018-09-28,117610.0,117620.0,117580.0,117600.0,23,48
12375,2018-09-28,117610.0,117690.0,117560.0,117670.0,23,49
